## RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [2]:
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf
  ✓ Loaded 149 pages

Total documents loaded: 149


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'Pdftools SDK', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-07-14T17:56:09+00:00', 'source': '..\\data\\pdf\\Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf', 'total_pages': 149, 'page': 0, 'page_label': '1', 'source_file': 'Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf', 'file_type': 'pdf'}, page_content='Sam Bhagwat\nCofounder & CEO/uni00A0Mastra.ai\nSam Bhagwat Principles of Building AI Agents\nPrinciples \nof Building \nAI Agents\n2nd EditionUnderstanding frontier tech is essential for building the \nfuture. Sam has done it once with Gatsby, and now again \nwith Mastra - Paul Klein, CEO of Browserbase\nRapid advances in large language models (LLMs) have \nmade new kinds of AI applications, known as agents, \npossible. Written by a veteran of web development, \nPrinciples of Building AI Agents focuses on the substance \nwithout hype or buzzwords. This book walks through:\n• The key building blocks of agen

In [4]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [5]:
chunks=split_documents(all_pdf_documents)
chunks

Split 149 documents into 151 chunks

Example chunk:
Content: Sam Bhagwat
Cofounder & CEO/uni00A0Mastra.ai
Sam Bhagwat Principles of Building AI Agents
Principles 
of Building 
AI Agents
2nd EditionUnderstanding frontier tech is essential for building the 
futur...
Metadata: {'producer': 'Pdftools SDK', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-07-14T17:56:09+00:00', 'source': '..\\data\\pdf\\Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf', 'total_pages': 149, 'page': 0, 'page_label': '1', 'source_file': 'Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Pdftools SDK', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-07-14T17:56:09+00:00', 'source': '..\\data\\pdf\\Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf', 'total_pages': 149, 'page': 0, 'page_label': '1', 'source_file': 'Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf', 'file_type': 'pdf'}, page_content='Sam Bhagwat\nCofounder & CEO/uni00A0Mastra.ai\nSam Bhagwat Principles of Building AI Agents\nPrinciples \nof Building \nAI Agents\n2nd EditionUnderstanding frontier tech is essential for building the \nfuture. Sam has done it once with Gatsby, and now again \nwith Mastra - Paul Klein, CEO of Browserbase\nRapid advances in large language models (LLMs) have \nmade new kinds of AI applications, known as agents, \npossible. Written by a veteran of web development, \nPrinciples of Building AI Agents focuses on the substance \nwithout hype or buzzwords. This book walks through:\n• The key building blocks of agen

## Embedding & VectorStoreDB 

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

d:\LangGraph Tutorials\Krish Naik Courses\YT\Building RAG Pipeline From Scratch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
class EmbeddingManager:
    """Handles document embedding generation using Sentence Transformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initializes the Embedding Manager
        
        Args:
            model_name: HugggingFace model name for sentence embedding
        """

        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Loads the SentenceTransformer Model"""

        try:
            print(f"Loading Embedding Model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model Loaded Successfully. Embedding Dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error Loading Model: {self.model_name}: {e}")
            raise
    
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate Embeddings for a list of text

        Args:
            texts: List of text strings to embed
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dimensions)
        """

        if not self.model:
            raise ValueError("Model Not Loaded")
        
        print(f"Generating Embeddings for {len(texts)} texts")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
    def get_sentence_embedding_dimension(self) -> int:
        """
        Get the Embedding Dimensions of the model

        Returns:
            integer value of the Embedding Dimensions
        """

        if not self.model:
            raise ValueError("Model not Loaded")
        return self.model.get_sentence_embedding_dimension()
    

## initialize the embedding manager

embedding_manager = EmbeddingManager()
embedding_manager

Loading Embedding Model: all-MiniLM-L6-v2
Model Loaded Successfully. Embedding Dimension: 384


### Vector Store DB

In [8]:
import os

In [23]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [24]:
chunks

[Document(metadata={'producer': 'Pdftools SDK', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-07-14T17:56:09+00:00', 'source': '..\\data\\pdf\\Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf', 'total_pages': 149, 'page': 0, 'page_label': '1', 'source_file': 'Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf', 'file_type': 'pdf'}, page_content='Sam Bhagwat\nCofounder & CEO/uni00A0Mastra.ai\nSam Bhagwat Principles of Building AI Agents\nPrinciples \nof Building \nAI Agents\n2nd EditionUnderstanding frontier tech is essential for building the \nfuture. Sam has done it once with Gatsby, and now again \nwith Mastra - Paul Klein, CEO of Browserbase\nRapid advances in large language models (LLMs) have \nmade new kinds of AI applications, known as agents, \npossible. Written by a veteran of web development, \nPrinciples of Building AI Agents focuses on the substance \nwithout hype or buzzwords. This book walks through:\n• The key building blocks of agen

In [25]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]
texts

['Sam Bhagwat\nCofounder & CEO/uni00A0Mastra.ai\nSam Bhagwat Principles of Building AI Agents\nPrinciples \nof Building \nAI Agents\n2nd EditionUnderstanding frontier tech is essential for building the \nfuture. Sam has done it once with Gatsby, and now again \nwith Mastra - Paul Klein, CEO of Browserbase\nRapid advances in large language models (LLMs) have \nmade new kinds of AI applications, known as agents, \npossible. Written by a veteran of web development, \nPrinciples of Building AI Agents focuses on the substance \nwithout hype or buzzwords. This book walks through:\n• The key building blocks of agents: \nproviders, models, prompts, tools, memory\n• How to break down complex tasks \nwith agentic workflows\n• Giving agents access to knowledge bases \nwith RAG (retrieval-augmented generation)\nIf you’re trying to build agents or assistants into your \nproduct, you need to read “Principles” ASAP - Peter \nKazanjy, Author of Founding Sales and CEO of Atrium\nSam Bhagwat is the foun

In [26]:
### Generate the embeddings

embeddings = embedding_manager.generate_embeddings(texts)

### Store it in the Vector DB

vectorstore.add_documents(chunks, embeddings)

Generating Embeddings for 151 texts


Batches: 100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


Generated embeddings with shape: (151, 384)
Adding 151 documents to vector store...
Successfully added 151 documents to vector store
Total documents in collection: 151


## Retrieval Pipeline for RAG

In [29]:
class RAGRetriever:
    """Handles query based retrieval from the vectore store"""

    def __init__(self, vector_store:VectorStore, embedding_manager:EmbeddingManager):
        """
        Initialize the Retriever

        Args:
            vector_store: Vector store for the embeddings
            embedding_manager: Manager for generating query embeddings
        """

        self.vector_store = vector_store
        self.embedding_manager = embedding_manager


    def retrieve(self, query: str, top_k: int=5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The Search Query
            top_k: Numberof top results to return
            score_threshold: Minimum similarity Score Threshold

        Returns:
            List of all the dictonaries combining retrieved documents and metadata
        """

        print(f"Retrieving documents for query: '{query}")
        print(f"Top K: {top_k}, Score Threshold: '{score_threshold}")

        query_embedding = self.embedding_manager.generate_embeddings([query])[0]


        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    similarity_score = 1 - distance
                    if similarity_score>=score_threshold:
                        retrieved_docs.append({
                            'id':doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i+1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No Documents Found")

            return retrieved_docs
        except Exception as e:
            print(f"Error: {e}")
            return []
        
rag_retriever = RAGRetriever(vectorstore, embedding_manager)
rag_retriever


In [30]:
rag_retriever.retrieve("When to use MCP")

Retrieving documents for query: 'When to use MCP
Top K: 5, Score Threshold: '0.0
Generating Embeddings for 1 texts


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.66it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_a02e189d_63',
  'content': "46 S A M  B H A G WAT \nConversely , if you want to create a client with \naccess to other MCP servers, here’ s how you would \ndo that :\nWhat’s next for MCP \nMCP as a protocol is technically impressive, but the \necosystem is still working to resolve a few \nchallenges :\nFirst, discovery. There's no centralized or stan- \ndardized way to ﬁnd MCP tools. While various \nregistries have popped up, this has created its own \nsort of fragmentation .\nIn April, we somewhat tongue-in-cheek built the",
  'metadata': {'source': '..\\data\\pdf\\Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf',
   'source_file': 'Principles of Building AI Agents by Sam Bhagwat - Mastra AI.pdf',
   'producer': 'Pdftools SDK',
   'creationdate': '',
   'total_pages': 149,
   'file_type': 'pdf',
   'moddate': '2025-07-14T17:56:09+00:00',
   'page': 60,
   'page_label': '61',
   'content_length': 489,
   'creator': 'PyPDF',
   'doc_index': 63},
  'similarit